In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
mp_drawing= mp.solutions.drawing_utils
mp_pose= mp.solutions.pose
mp_drawing_styles = mp.solutions.drawing_styles


In [3]:
def calculte_angle(a,b,c):
    """
    This Function calculates the angle between 3 landmarks.
    """
    a= np.array(a)
    b= np.array(b)
    c= np.array(c)
    radians= np.arctan2(a[1]-b[1],a[0]-b[0]) - np.arctan2(c[1]-b[1],c[0]-b[0])
    deg= np.abs(np.degrees(radians))
    if deg >180:
        deg = 360- deg
    return np.round(deg,2)

In [6]:
cap = cv2.VideoCapture(0)
counter_l = 0
stage_l= None
counter_r = 0
stage_r= None
#Mediapipe Instance:
with mp_pose.Pose(min_tracking_confidence=0.5,min_detection_confidence=0.5) as pose:
    while cap.isOpened():
        ret, image = cap.read()
        image.flags.writeable = False
        # Recolor image to RGB:
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        # Processing the image:
        results= pose.process(image)
        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #Extract Landmarks:
        try:
            landmarks= results.pose_landmarks.landmark
            #Get coordinates:
            shoulder_l= [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            elbow_l= [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y]
            wrist_l= [landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y]
            shoulder_r= [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            elbow_r= [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW].y]
            wrist_r= [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST].y]
            # Get the angle:
            angle_l= calculte_angle(shoulder_l,elbow_l,wrist_l)
            angle_r= calculte_angle(shoulder_r,elbow_r,wrist_r)
            # Visualize angle:
            #cv2.putText(image, str(angle),tuple(np.multiply(elbow, [640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255) , 2, cv2.LINE_AA)
            # Curl Counter Logic:
            if angle_l > 160:
                stage_l= "Down"
            if angle_l < 45 and stage_l== "Down":
                stage_l ="Up"
                counter_l+=1
                
            if angle_r > 160:
                stage_r= "Down"
            if angle_r < 45 and stage_r== "Down":
                stage_r ="Up"
                counter_r+=1
            # Status Box:
            cv2.rectangle(image, (0,0),(235, 75),(245,117,0),-1)
            cv2.putText(image, "Right Hand Counter",(80,60),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255) , 2, cv2.LINE_AA )
            cv2.putText(image, str(counter_r),(10,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255) , 2, cv2.LINE_AA )
            
            cv2.rectangle(image, (400,0),(635, 75),(245,117,0),-1)
            cv2.putText(image, "Left Hand Counter",(420,60),cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,255,255) , 2, cv2.LINE_AA )
            cv2.putText(image, str(counter_l),(580,60),cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255) , 2, cv2.LINE_AA )
        except:
            pass

        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()) 
        # Flip the image horizontally for a selfie-view display.
        cv2.imshow('Pose Estimation', image)
        #Press esc key on keyboard to exit:
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()